In [ ]:
#!pip install tensorflow_text==2.5.0.
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import mode
import re
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import KFold

In [ ]:
twitter = pd.read_csv("labeled_tweet_table.csv")
twitter = twitter[twitter["Race"].notna()]
twitter = twitter[twitter["Race"] != 5]
twitter["Race"] -= 1
twitter["Race"].value_counts()

In [ ]:
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        try:
            t = re.sub(regx, regexMap[regx], t)
        except TypeError:
            continue
    return t
twitter["Tweet"] = twitter["Tweet"].apply(preprocess)
twitter

In [ ]:
names = twitter["Name"].unique()
labels = []
tweet_dict = {}
for name in names:
    tweet_dict[name] = twitter[twitter["Name"] == name]["Tweet"].tolist()

In [ ]:
label_dict = twitter.set_index("Name").to_dict()["Race"]

In [ ]:
concatenated_tweets = []
labels = []
for name in names:
    concat = ''.join(str(tweet_dict[name]))
    concatenated_tweets.append(concat)
    labels.append(label_dict[name])

concat_df = pd.DataFrame({"User": np.array(names), "Tweet": concatenated_tweets, "Race": np.array(labels)})
len(names), len(concatenated_tweets), len(labels)

In [ ]:
concat_df

In [ ]:
def constructModel():
    bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
    text_input = keras.layers.Input(shape=(), dtype=tf.string)
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)
    
    hidden_layer = keras.layers.Dense(100, activation='relu')(outputs["pooled_output"])

    dense = keras.layers.Dense(4, activation='softmax')(hidden_layer)

    model = keras.Model(inputs=[text_input], outputs=[dense])
    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    model.summary()
    return model

model = constructModel()

In [ ]:
accs = []
reports = []
confusion_matrices = []

kf = KFold(n_splits = 5)
i = 1
for train, test in kf.split(concat_df):
    print("Fold %d:" % i)
    print()
    i += 1
    train_df = concat_df.iloc[train]
    test_df = concat_df.iloc[test]
    
    x_train, y_train = train_df["Tweet"], train_df["Race"]
    x_test, y_test = test_df["Tweet"], test_df["Race"]
    
    model.fit(x_train, y_train, epochs = 5)
    y_pred = np.argmax(model.predict(x_test), axis=1)
    acc = accuracy_score(y_test, y_pred)
    accs.append(acc)
    print("Accuracy: ", acc)
    print()
    report = classification_report(y_test, y_pred)
    print(report)
    print()
    reports.append(report)
    cm = confusion_matrix(y_test, y_pred, normalize='true')
    print(cm)
    print()
    confusion_matrices.append(cm)

In [ ]:
np.average(accs)

In [ ]:
sum(confusions) / 5